In [1]:
import numpy as np
from keras.datasets import imdb
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from keras_preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.layers import BatchNormalization

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=5000)
print(len(x_test))
x_train = pad_sequences(x_train, maxlen=100)
x_test = pad_sequences(x_test, maxlen=100)
print(len(x_test))

25000
25000


In [4]:
model = Sequential()
model.add(Embedding(max_features, 50, input_length=100))
model.add(SpatialDropout1D(0.3))
#model.add(BatchNormalization())
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
batch_size = 2048
epochs = 10
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)

Epoch 1/10
13/13 [==============================] - 11s 618ms/step - loss: 0.6916 - accuracy: 0.5609 - val_loss: 0.6865 - val_accuracy: 0.6402
Epoch 2/10
13/13 [==============================] - 8s 604ms/step - loss: 0.6393 - accuracy: 0.6898 - val_loss: 0.5446 - val_accuracy: 0.7459
Epoch 3/10
13/13 [==============================] - 8s 592ms/step - loss: 0.5007 - accuracy: 0.7655 - val_loss: 0.4418 - val_accuracy: 0.7909
Epoch 4/10
13/13 [==============================] - 8s 655ms/step - loss: 0.4024 - accuracy: 0.8234 - val_loss: 0.3745 - val_accuracy: 0.8330
Epoch 5/10
13/13 [==============================] - 9s 668ms/step - loss: 0.3476 - accuracy: 0.8512 - val_loss: 0.3591 - val_accuracy: 0.8425
Epoch 6/10
13/13 [==============================] - 9s 678ms/step - loss: 0.3119 - accuracy: 0.8692 - val_loss: 0.3437 - val_accuracy: 0.8487
Epoch 7/10
13/13 [==============================] - 8s 637ms/step - loss: 0.2984 - accuracy: 0.8780 - val_loss: 0.3629 - val_accuracy: 0.8454
Epoch

In [6]:
scores = model.evaluate(x_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

782/782 [==============================] - 37s 47ms/step - loss: 0.3460 - accuracy: 0.8493
Accuracy: 84.93%
